In [ ]:
from __future__ import print_function

from depmapomics.config import *

from depmapomics import loading, tracker
from depmapomics import terra as myterra
from depmapomics import mutations as omics_mut
from depmapomics import copynumbers as omics_cn
from genepy import terra
from genepy.utils import helper as h
from gsheets import Sheets
from depmapomics import tracker as track
from genepy.utils import helper as h

import pandas as pd
import dalmatian as dm
from bokeh.plotting import output_notebook

%load_ext autoreload
%autoreload 2

output_notebook()

In [ ]:
sheets = Sheets.from_files(MY_ID, MYSTORAGE_ID)

In [ ]:
from taigapy import TaigaClient
tc = TaigaClient()

depmap_master = tc.get(name=DEPMAP_TAIGA, file="ACH").set_index(
    "arxspan_id", drop=True
)

In [ ]:
depmap_master[depmap_master['Original Source'].isnull()].index

In [ ]:
depmap_master.loc['ACH-000114', ]

In [ ]:
# check if samples on tracker that have original source agree with master
t = tracker.getTracker()
withos = t[~t.original_source.isnull()]

In [ ]:
for arxs in withos.arxspan_id.unique():
    i = withos[withos.arxspan_id == arxs].index[0]
    if withos.loc[i, 'original_source'] != depmap_master.loc[arxs, 'Original Source'] and withos.loc[i, 'original_source'] != 'HSSRB':
        if depmap_master.loc[arxs, 'Original Source'] == None:
            print('No annotation on master: ' + arxs)
        else:
            print(arxs + ':  tracker: ' + withos.loc[i, 'original_source'] + ", master: " + depmap_master.loc[arxs, 'Original Source'])

In [ ]:
len(withos.arxspan_id.unique())

In [ ]:
# rename HSSRB into HSRRB
for i in t[t.original_source == 'HSSRB'].index:
    t.loc[i, 'original_source'] = 'HSRRB'
    print(i)

In [ ]:
from genepy.google.google_sheet import dfToSheet
dfToSheet(t, 'ccle sample tracker', secret="../../.credentials.json")